# Cubic Spline Interpolation

In [50]:
from sympy import *
import numpy as np
import math

In [81]:
f=[]
a=1
b=1
x = Symbol('x')
A = symbols('a0:5')
dx = diff((a*x**2) + b , x)
print(dx)
print(A[0])
dA0 = diff(A[0]*exp(A[0]), A[0])
dA0
exp(x).evalf(subs={x:1})
exp(1).evalf(5)

2*x
a0


2.7183

## Natural Cubic Spline

In [106]:



def natural_cubic_spline(N, data_points, a):
    S = lambda x, xj, a, b, c, d : a + (b*(x-xj)) + (c*(x-xj)**2) + ((d*(x-xj)**3))
    h=np.array([data_points[i+1][0]-data_points[i][0] for i in range(N-1)])
    alpha = np.zeros(N-1)
    for i in range(N-1):
        alpha[i] = 3*(a[i+1]-a[i])/h[i] - 3*(a[i]-a[i-1])/h[i-1]

In [123]:
#example 2 pg 150
precision = 5
data_points=np.array([[0,1],
                      [1, exp(1).evalf(precision)],
                      [2, exp(2).evalf(precision)],
                      [3, exp(3).evalf(precision)]]
                    )
N = len(data_points)
f = lambda c : exp(c).evalf(precision)
a = np.array([data_points[i][1] for i in range(len(data_points))])

# init
b = np.zeros(N)
c = np.zeros(N)
d = np.zeros(N)
S = lambda x, xj, a, b, c, d : a + (b*(x-xj)) + (c*(x-xj)**2) + ((d*(x-xj)**3))

# step 1
h=np.array([data_points[i+1][0]-data_points[i][0] for i in range(N-1)])
# step 2
alpha = np.zeros(N-1)
for i in range(N-1):
    alpha[i] = 3*(a[i+1]-a[i])/h[i] - 3*(a[i]-a[i-1])/h[i-1]
# step 3
l = np.zeros(N)
miu = np.zeros(N)
z = np.zeros(N)
l[0] = 1
miu[0] = 0
z[0] = 0
# step 4
for i in range(1, N-1):
    l[i] = 2*(data_points[i+1][0] - data_points[i-1][0]) - (h[i-1]*miu[i-1])
    miu[i] = h[i]/l[i]
    z[i] = (alpha[i]-(h[i-1]*z[i-1]))/l[i]
# step 5
l[N-1] = 1
z[N-1] = 0
c[N-1] = 0

print(l)
print(miu)
print(z)

[1.   4.   3.75 1.  ]
[0.         0.25       0.26666667 0.        ]
[0.         2.21437073 5.83007406 0.        ]
